# Cryptohack 
## [ELLIPTIC CURVES](https://cryptohack.org/courses/elliptic/course_details/)

Writeup by Jupyter notebook

<img src="img/スクリーンショット 2023-05-04 14.13.32.png" alt="official">
crypto{Abelian}

任意の点 P に関して、 P + O = O + P = P

O 以外の2点 P(x_p, y_p), Q(x_q, y_q) に関して、 x_p = x_q かつ y_p = -y_q のとき、 P + Q = O

```
print(-6936%9739)
# 2803
```
crypto{8045,2803}

Algorithm for the addition of two points: P + Q

(a) If P = O, then P + Q = Q.<br>
(b) Otherwise, if Q = O, then P + Q = P.<br>
(c) Otherwise, write P = (x1, y1) and Q = (x2, y2).<br>
(d) If x1 = x2 and y1 = −y2, then P + Q = O.<br>
(e) Otherwise:<br>
&emsp;(e1) if P ≠ Q: λ = (y2 - y1) / (x2 - x1)<br>
&emsp;(e2) if P = Q: λ = (3x12 + a) / 2y1<br>
(f) x3 = λ2 − x1 − x2,     y3 = λ(x1 −x3) − y1<br>
(g) P + Q = (x3, y3)<br>


In [2]:
def ec_add(p, a, P, Q):
    zero = [0, 0]

    if P[0] == Q[0] and P[1] == -Q[1] % p:
        return zero
    elif P == zero:
        return Q
    elif Q == zero:
        return P
    elif Q == P:
        lamb = (pow((2*P[1]), -1, p) * (3*(P[0]**2) + a)) % p
    else:
        lamb = (pow(Q[0]-P[0], -1, p)*(Q[1]-P[1])) % p
    x = (lamb**2 - P[0] - Q[0]) % p
    # Q = (lamb**2 - Q1 - Q2) % p
    return [x, (lamb*(P[0]-x) - P[1]) % p]


p = 9739
a = 497
b = 1768
P = [493, 5564]
Q = [1539, 4742]
R = [4403, 5202]
print(ec_add(p, a, ec_add(p, a, P, P), ec_add(p, a, Q, R)))


[4215, 2162]


Double and Add algorithm for the scalar multiplication of point P by n

Input: P in E(Fp) and an integer n > 0<br>
1. Set Q = P and R = O.<br>
2. Loop while n > 0.<br>
&emsp;3. If n ≡ 1 mod 2, set R = R + Q.<br>
&emsp;4. Set Q = 2 Q and n = ⌊n/2⌋.<br>
&emsp;5. If n > 0, continue with loop at Step 2.<br>
6. Return the point R, which equals nP.<br>

In [64]:
import copy


def ec_scalar(p, a, P, n):
    Q = copy.deepcopy(P)
    R = [0, 0]
    while n > 0:
        if n % 2 == 1:
            R = ec_add(p, a, R, Q)
        Q = ec_add(p, a, Q, Q)
        n = n // 2
    return R


p = 9739
a = 497
b = 1768
n = 7863
P = [2339, 2213]
print(ec_scalar(p, a, P, n))


[9467, 2742]


In [87]:
from math import gcd
import hashlib

# ルジャンドル記号を実装。平方剰余かを判断する。
# pを法とする時、a^2と合同の整数があるか判断する。
def legendre_symbol(a, p):
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# 0~(p-1)の間でpを法とする時、a^2と合同の整数のリストを作る
def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False


def ec(p, a, b, x):
    roots = SquareRoots(p)
    y = checkRoot(roots, (x**3+a*x+b) % p)
    return y


p = 9739
a = 497
b = 1768
G = [1804, 5368]
QA = [815, 3190]
nB = 1829

sha1 = hashlib.sha1()
data = ec_scalar(p, a, QA, nB)
print(data)
sha1.update(str(data[0]).encode("utf-8"))
print(sha1.hexdigest())


[7929, 707]
80e5212754a824d3a4aed185ace4f9cac0f908bf


In [5]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii', 'ignore'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii', 'ignore')
    else:
        return plaintext.decode('ascii', 'ignore')


def ec(p, a, b, x):
    roots = SquareRoots(p)
    y = checkRoot(roots, (x**3+a*x+b) % p)
    return y

p = 9739
a = 497
b = 1768
G = [1804, 5368]
q_x = 4726
nB = 6534

data = str(ec_scalar(p, a, [q_x, ec(p, a, b, q_x)], nB)[0])
iv = 'cd9da9f1c60925922377ea952afc212c'
ciphertext = 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'

print(decrypt_flag(data, iv, ciphertext))


crypto{3ff1c1ent_k3y_3xch4ng3}


In [113]:
# BY^2 = X^3 + AX^2 + X (Mod p)
# Montgomery curve
def tonelli_shanks(n, p):
    """
    Computes the square root of n modulo p using the Tonelli-Shanks algorithm.
    Assumes that p is a prime and n is a quadratic residue modulo p.
    """
    # Step 1: Compute q and s such that p-1 = q * 2^s
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1

    # Step 2: Find a non-residue mod p by brute force
    z = 1
    while legendre_symbol(z, p) != -1:
        z += 1

    # Step 3: Initialize variables
    m = s
    c = pow(z, q, p)
    t = pow(n, q, p)
    r = pow(n, (q+1)//2, p)

    # Step 4: Main loop
    while t != 1:
        # Find the smallest i such that t^(2^i) = 1
        i = 0
        temp = t
        while temp != 1:
            temp = (temp * temp) % p
            i += 1

        # Update variables
        b = pow(c, 1 << (m-i-1), p)
        m = i
        c = (b * b) % p
        t = (t * c) % p
        r = (r * b) % p

    return r


def MontgomeryCurve(p, a, b, x):  # perfect!
    y = (pow(b, -1, p)*(x**3 + a*(x**2) + x) % p) % p
    if p % 4 == 1:
        return tonelli_shanks(y, p)
    return False


def check_point(p, a, b, P):
    if (pow(P[1], 2, p) - pow(P[0], 3, p)-a*pow(P[0], 2, p)-P[0]) % p == 0:
        return True
    return False


def MontgomeryCurve_add(p, a, b, P, Q):
    zero = [0, 0]
    if P[0] == Q[0] and P[1] == -Q[1] % p:
        return zero
    elif P == zero:
        return Q
    elif Q == zero:
        return P
    else:
        lamb = (pow(Q[0]-P[0], -1, p)*(Q[1]-P[1])) % p
        x = (b*pow(lamb, 2, p)-a-P[0]-Q[0]) % p
        S = [x, (lamb*(P[0]-x)-P[1]) % p]
        if check_point(p, a, b, S):
            return S
        else:
            print("Add is not correct point")


def MontgomeryCurve_double(p, a, b, P):
    lamb = (pow(2*b*P[1], -1, p)*(3*(P[0]**2)+2*a*P[0]+1)) % p
    x = b*(lamb**2)-a-2*P[0] % p
    S = [x, (lamb*(P[0]-x)-P[1]) % p]
    if check_point(p, a, b, S):
        return S
    else:
        print("Double is not correct point")


def MontgomeryCurve_scalar(p, a, b, k, P):
    Q = copy.deepcopy(P)
    R = [0, 0]
    while k > 0:
        if k % 2 == 1:
            R = MontgomeryCurve_add(p, a, b, R, Q)
        Q = MontgomeryCurve_double(p, a, b, Q)
        k = k // 2
    return R


a = 486662
b = 1
p = 2**255 - 19
Gx = 9
Gy = ((MontgomeryCurve(p, a, b, Gx)))
Q = MontgomeryCurve_scalar(p, a, b, int(0x1337c0decafe), [Gx, Gy])
print(f'crypto{{{Q[0]}}}')


crypto{49231350462786016064336756977412654793383964726771892982507420921563002378152}


### output.txt
Point(x=280810182131414898730378982766101210916, y=291506490768054478159835604632710368904)

{'iv': '07e2628b590095a5e332d397b8a59aa7', 'encrypted_flag': '8220b7c47b36777a737f5ef9caa2814cf20c1c1ef496ec21a9b4833da24a008d0870d3ac3a6ad80065c138a2ed6136af'}

In [114]:
from Crypto.Cipher import AES
from Crypto.Util.number import inverse
from Crypto.Util.Padding import pad, unpad
from collections import namedtuple
from random import randint
import hashlib
import os

# Create a simple Point class to represent the affine points.
Point = namedtuple("Point", "x y")

# The point at infinity (origin for the group law).
O = 'Origin'

FLAG = b'crypto{??????????????????????????????}'


def check_point(P: tuple):
    if P == O:
        return True
    else:
        return (P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p


def point_inverse(P: tuple):
    if P == O:
        return P
    return Point(P.x, -P.y % p)


def point_addition(P: tuple, Q: tuple):
    # based of algo. in ICM
    if P == O:
        return Q
    elif Q == O:
        return P
    elif Q == point_inverse(P):
        return O
    else:
        if P == Q:
            lam = (3*P.x**2 + a)*inverse(2*P.y, p)
            lam %= p
        else:
            lam = (Q.y - P.y) * inverse((Q.x - P.x), p)
            lam %= p
    Rx = (lam**2 - P.x - Q.x) % p
    Ry = (lam*(P.x - Rx) - P.y) % p
    R = Point(Rx, Ry)
    assert check_point(R)
    return R


def double_and_add(P: tuple, n: int):
    # based of algo. in ICM
    Q = P
    R = O
    while n > 0:
        if n % 2 == 1:
            R = point_addition(R, Q)
        Q = point_addition(Q, Q)
        n = n // 2
    assert check_point(R)
    return R


def gen_shared_secret(Q: tuple, n: int):
    # Bob's Public key, my secret int
    S = double_and_add(Q, n)
    return S.x


def encrypt_flag(shared_secret: int):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Encrypt flag
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(FLAG, 16))
    # Prepare data to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_flag'] = ciphertext.hex()
    return data


# Define the curve
p = 310717010502520989590157367261876774703
a = 2
b = 3

# Generator
g_x = 179210853392303317793440285562762725654
g_y = 105268671499942631758568591033409611165
G = Point(g_x, g_y)

# My secret int, different every time!!
n = randint(1, p)

# Send this to Bob!
public = double_and_add(G, n)
print(public)

# Bob's public key
b_x = 272640099140026426377756188075937988094
b_y = 51062462309521034358726608268084433317
B = Point(b_x, b_y)

# Calculate Shared Secret
shared_secret = gen_shared_secret(B, n)

# Send this to Bob!
ciphertext = encrypt_flag(shared_secret)
print(ciphertext)

Point(x=154532252687400881319544377422941445289, y=44855815323134229342730882910765465500)
{'iv': 'ca377ddbe0385e697fb0dcc573ecb7e4', 'encrypted_flag': '8589451e58f6339f31464314b6ee6fd127032e8291201a6f61875614efb6e66d4d444230b3a4e074b45b8da54a8e5d2d'}


In [ ]:
def baby_step_giant_step(p, a, b, P, Q):
    m = int(E.order() ^ 0.5 + 1)

    baby = []
    b = Q
    for j in range(m):
        baby.append(b)
        b -= P

    giant = m * P
    for i in range(1, m):
        if giant in baby:
            d = i*m + baby.index(giant)
            return d
        else:
            giant += (m * P)
    print "not found"
    return -1


p = 229
a = 1
b = 44

E = EllipticCurve(GF(p), [a, b])
P = E([5, 116])
Q = E([155, 166])

d = 176

assert Q == d * P

print(d)
print(baby_step_giant_step(E, P, Q))

b_x = 272640099140026426377756188075937988094
b_y = 51062462309521034358726608268084433317

x = 280810182131414898730378982766101210916
y = 291506490768054478159835604632710368904

# Define the curve
p = 310717010502520989590157367261876774703
a = 2
b = 3

# Generator
g_x = 179210853392303317793440285562762725654
g_y = 105268671499942631758568591033409611165
G = Point(g_x, g_y)

X = 3
Y = 193
MOD = 10**9 + 7
r = solve(X, Y, MOD)
assert pow(X, r, MOD) == Y, (pow(X, r, MOD), Y)
print(solve(g_x, ))


In [ ]:
def baby_step_giant_step(E, P, Q):
    m = int(E.order()^0.5 + 1)

    baby = []
    b = Q
    for j in range(m):
        baby.append(b)
        b -= P

    giant = m * P
    for i in range(1, m):
        if giant in baby:
            d = i*m + baby.index(giant)
            return d
        else:
            giant += (m * P)
    print "not found"
    return -1

p = 229
a = 1
b = 44

E = EllipticCurve(GF(p), [a, b])
P = E([5, 116])
Q = E([155, 166])

d = 176

assert Q == d * P

print(d)
print(baby_step_giant_step(E, P, Q))

b_x = 272640099140026426377756188075937988094
b_y = 51062462309521034358726608268084433317

x=280810182131414898730378982766101210916
y=291506490768054478159835604632710368904

# Define the curve
p = 310717010502520989590157367261876774703
a = 2
b = 3

# Generator
g_x = 179210853392303317793440285562762725654
g_y = 105268671499942631758568591033409611165
G = Point(g_x, g_y)

X = 3; Y = 193; MOD = 10**9 + 7
r = solve(X, Y, MOD)
assert pow(X, r, MOD) == Y, (pow(X, r, MOD), Y)
print(solve(g_x, ))




In [ ]:
import copy
from math import gcd

def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False


def ec_get_0(p, a, P):
    return ec_add(p, a, P, [P[0], -P[1] % p])


def ec_scalar(p, a, P, n):
    Q = copy.deepcopy(P)
    R = ec_get_0(p, a, P)
    while n > 0:
        if n % 2 == 1:
            R = ec_add(p, a, R, Q)
        Q = ec_add(p, a, Q, Q)
        
        n = n // 2
    return R


p = 9739
a = 497
b = 1768
n = 1337
X = [5323, 5438]
P = [2339, 2213]
# print(ec_get_o(p,a,b,P))
print(X)
print(ec_add(p, a, X, ec_get_0(p, a, X)))
print(ec_scalar(p, a, X, n))


In [ ]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N

def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i,2,p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])    
    return square_list
def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False
# 楕円曲線の足し算
def ec_add(p, squares, x1, y1, x2, y2):
    if x1 != x2:
        lamb = (y2-y1)/(x2-x1)
    else:
        lamb = (3*(x1**2) + 9)/ (2*y1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 61
x1 = 5
y1 = 7
a = 9
b = 1
x2=0
squares = SquareRoots(p)
print(ec_add(p, squares ,x1,y1,x1,y1))
print(ec_add(p, squares ,x1,y1,26,50))

while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b)%p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            print(x3, y3, x2, y2)
            if x3 == 0 and y3 == 0:
                print(x2, y2)
                break
    x2 += 1


In [ ]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N


def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i, 2, p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])
    return square_list


def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False

# 楕円曲線の足し算


def ec_add(p, x1, y1, x2, y2):
    if x1 != x2:
        lamb = (y2-y1)/(x2-x1)
    else:
        lamb = (3*(x1**2) + 9) / (2*y1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 9739
x1 = 8045
y1 = 6936
a = 497
b = 1768
x2 = 0
squares = SquareRoots(p)
x0 = 0
y0 = checkRoot(squares, (x0**3+a*x0+b) % p)
print(x0,y0)
while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b) % p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            if x3 == x0 and y3 == y0:
                print(x2, y2)
                break
    x2 += 1


In [ ]:
from math import gcd

# 平方剰余


def legendre_symbol(a, p):
    """
    Computes the Legendre symbol (a|p).
    """
    if gcd(a, p) != 1:
        return 0
    elif pow(a, (p-1)//2, p) == p-1:
        return -1
    else:
        return 1

# Multiplicative Inverses in 𝔽p


def Inverses(p):
    inv_list = [0, 0]
    for i in range(p):
        for j in range(p):
            if (i*j) % p == 1:
                inv_list.append([i, j])
    return inv_list

# Square Roots Modulo N

def SquareRoots(p):
    square_list = [[0, 0]]
    for i in range(p):
        x = pow(i,2,p)
        if legendre_symbol(x, p) == 1:
            square_list.append([x, i])    
    return square_list
def checkRoot(list, a):
    try:
        ans = list[[g[0] for g in list].index(a)][1]
        return ans
    except:
        return False
# 楕円曲線の足し算
def ec_add(p, x1, y1, x2, y2):
    lamb = (y2-y1)/(x2-x1)
    x = (lamb**2 - x1 - x2) % p
    # y = (lamb**2 - y1 - y2) % p
    return x, (lamb*(x1-x) - y1) % p


p = 61
x1 = 8
y1 = 6936
a = 497
b = 1768
a = b = x2 = 0
x2=2
squares = SquareRoots(p)
while x2 < p:
    if x1 != x2:
        y2 = checkRoot(squares, (x2**3+a*x2+b)%p)
        if y2:
            x3, y3 = ec_add(p, x1, y1, x2, y2)
            print(x3, y3, x2, y2)
            if x3 == 0 and y3 == 0:
                print(x2, y2)
                break
    x2 += 1


In [ ]:
p=93
for i in range(p):
    print(i, 3**i % 93) 

In [26]:
print(9739%4)

3


In [ ]:
# 基本的にここでassertionエラーは出ないはず
def check_sqrt(x, n, p):
    assert(pow(x, 2, p) == n % p)

def modular_sqrt(n, p):
    if p % 4 == 3:
        x = pow(n, (p + 1) // 4, p)
        check_sqrt(x, n, p)
        return [x, p - x]
    else:
        # これから説明
        # Step 1.
        q, s = p - 1, 0
        while q % 2 == 0:
            q //= 2
            s += 1
        # Step 2.
        z = 2
        while legendre_symbol(z, p) != -1:
            z += 1
        m, c, t, r = s, pow(z, q, p), pow(n, q, p), pow(n, (q + 1) // 2, p)
        # Step 4.
        while t != 1:
            pow_t = pow(t, 2, p)
            for j in range(1, m):
                if pow_t == 1:
                    m_update = j
                    break
                pow_t = pow(pow_t, 2, p)
            b = pow(c, int(pow(2, m - m_update - 1)), p)
            m, c, t, r = m_update, pow(b, 2, p), t * pow(b, 2, p) % p, r * b % p

        # 答えの確認
        check_sqrt(r, n, p)
        return [r, p - r]